# Wildfire Risk - Ingest - Athena Table Setup
__Team 3 - Dave Friesen, John Chen, and Kyle Dalope__<br>
__ADS-508-02-SP23__<br><br>
__GitHub link: https://github.com/davefriesen/wildfire-risk__

In [1]:
__authors__ = ['Dave Friesen', 'John Chen', 'Kyle Dalope']
__contact__ = ['dfriesen@sandiego.edu', 'johnchen@sandiego.edu', 'kdalope@sandiego.edu']
__date__ = '2023-03-20'
__license__ = 'MIT'
__version__ = '1.0.1'

# Setup Basics

In [2]:
# Import basic libraries
import boto3
import sagemaker

# Import data access libraries
import pandas as pd
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

# Import utility libraries
from IPython.core.display import display, HTML

In [3]:
# Establish session fundamentals
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.Session().client(service_name='sagemaker', region_name=region)

# Register S3 CSVs with Athena

In [4]:
ingest_create_athena_table_passed = False

In [5]:
%store -r ingest_create_athena_db_passed
try:
    ingest_create_athena_db_passed
    print(ingest_create_athena_db_passed)
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS NOTEBOOK *********************************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")
    
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")    

True
[OK]


In [6]:
%store -r s3_private_path
try:
    s3_private_path
    print(s3_private_path)
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS NOTEBOOK *********************************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

s3://sagemaker-us-east-1-074876746575/widfire-risk/csv/


In [7]:
# Set Athena database name
database_name = 'dsoaws'

# Set S3 staging directory (temp directory for Athena queries)
s3_staging_dir = "s3://{0}/athena/staging/".format(bucket)

# Establish S3 connection
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

## Register fires.csv as Athena table

In [8]:
fires_csv = 'fires.csv'
fires_tb = 'fires'

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    ContainmentDateTime date,
    ControlDateTime date,
    DiscoveryAcres float,
    EstimatedCostToDate date,
    FinalAcres float,
    FireBehaviorGeneral string,
    FireBehaviorGeneral1 string,
    FireBehaviorGeneral2 string,
    FireBehaviorGeneral3 string,
    FireCause string,
    FireCauseGeneral string,
    FireCauseSpecific string,
    FireDiscoveryDateTime date,
    FireOutDateTime date,
    GACC string,
    IncidentName string,
    IncidentShortDescription string,
    InitialLatitude float,
    InitialLongitude float,
    IsFireCauseInvestigated int,
    IsTrespass int,
    POOCity string,
    POOState string,
    PredominantFuelModel string,
    PrimaryFuelModel string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, fires_tb, s3_private_path+fires_csv
)
print(statement)

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws.fires(
    ContainmentDateTime date,
    ControlDateTime date,
    DiscoveryAcres float,
    EstimatedCostToDate date,
    FinalAcres float,
    FireBehaviorGeneral string,
    FireBehaviorGeneral1 string,
    FireBehaviorGeneral2 string,
    FireBehaviorGeneral3 string,
    FireCause string,
    FireCauseGeneral string,
    FireCauseSpecific string,
    FireDiscoveryDateTime date,
    FireOutDateTime date,
    GACC string,
    IncidentName string,
    IncidentShortDescription string,
    InitialLatitude float,
    InitialLongitude float,
    IsFireCauseInvestigated int,
    IsTrespass int,
    POOCity string,
    POOState string,
    PredominantFuelModel string,
    PrimaryFuelModel string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-074876746575/widfire-risk/csv/fires.csv'
TBLPROPERTIES ('skip.header.line.count'='1')


""


## Register weather.csv as Athena table

In [9]:
weather_csv = 'weather.csv'
weather_tb = 'weather'

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    Station string,
    WDate string,
    Latitude float,
    Longitude float,
    Elevation float,
    Name string,
    CDSD float,
    CDSD_attributes string,
    CLDD float,
    CLDD_attributes string,
    DT00 float,
    DT00_attributes string,
    DT32 float,
    DT32_attributes string,
    DX32 float,
    DX32_attributes string,
    DX70 float,
    DX70_attributes string,
    DX90 float,
    DX90_attributes string,
    EMNT float,
    EMNT_attributes string,
    EMXT float,
    EMXT_attributes string,
    HDSD float,
    HDSD_attributes string,
    HTDD float,
    HTDD_attributes string,
    TAVG float,
    TAVG_attributes string,
    TMAX float,
    TMAX_attributes string,
    TMIN float,
    TMIN_attributes string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, weather_tb, s3_private_path+weather_csv
)
print(statement)

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws.weather(
    Station string,
    WDate string,
    Latitude float,
    Longitude float,
    Elevation float,
    Name string,
    CDSD float,
    CDSD_attributes string,
    CLDD float,
    CLDD_attributes string,
    DT00 float,
    DT00_attributes string,
    DT32 float,
    DT32_attributes string,
    DX32 float,
    DX32_attributes string,
    DX70 float,
    DX70_attributes string,
    DX90 float,
    DX90_attributes string,
    EMNT float,
    EMNT_attributes string,
    EMXT float,
    EMXT_attributes string,
    HDSD float,
    HDSD_attributes string,
    HTDD float,
    HTDD_attributes string,
    TAVG float,
    TAVG_attributes string,
    TMAX float,
    TMAX_attributes string,
    TMIN float,
    TMIN_attributes string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-074876746575/widfire-risk/csv/weather.csv'
TBLPROPERTIES ('skip.header.line.count'='1')


""


## Verify table creation

In [10]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)

if [fires_tb, weather_tb] in df_show.values:
    ingest_create_athena_table_passed = True
%store ingest_create_athena_table_passed

df_show.head(5)

Stored 'ingest_create_athena_table_passed' (bool)


,tab_name
0,amazon_reviews_tsv
1,fires
2,weather


# Run Sample Queries

In [11]:
statement = """SELECT * FROM {}.{}
    WHERE FireCause = 'Natural' LIMIT 50""".format(
    database_name, fires_tb
)
print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM dsoaws.fires
    WHERE FireCause = 'Natural' LIMIT 50


,containmentdatetime,controldatetime,discoveryacres,estimatedcosttodate,finalacres,firebehaviorgeneral,firebehaviorgeneral1,firebehaviorgeneral2,firebehaviorgeneral3,firecause,...,incidentname,incidentshortdescription,initiallatitude,initiallongitude,isfirecauseinvestigated,istrespass,poocity,poostate,predominantfuelmodel,primaryfuelmodel


In [12]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

++++++++++++++++++++++++++++++++++++++++++++++++++++++
[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.
++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Review the New Athena Tables in the Glue Catalog

In [13]:
display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

# Store Variables and Close Session

In [14]:
# Store variables for subsequent notebooks
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed                    -> True
ingest_create_athena_table_passed                 -> True
ingest_create_athena_table_tsv_passed             -> True
s3_private_path                                   -> 's3://sagemaker-us-east-1-074876746575/widfire-ris
s3_private_path_tsv                               -> 's3://sagemaker-us-east-1-074876746575/amazon-revi
s3_public_path                                    -> 's3://wildfire-risk/'
s3_public_path_tsv                                -> 's3://amazon-reviews-pds/tsv'
setup_dependencies_passed                         -> True
setup_iam_roles_passed                            -> True
setup_instance_check_passed                       -> True
setup_s3_bucket_passed                            -> True


In [15]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [16]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>